In [1]:
import tensorflow as tf
import os

###Loading The Data

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ravirajsinh45/real-life-industrial-dataset-of-casting-product")

print("Path to dataset files:", path)

100%|██████████| 100M/100M [00:00<00:00, 161MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product/versions/2


In [3]:
path_to_dataset = "/root/.cache/kagglehub/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product/versions/2"

os.listdir(path_to_dataset)

['casting_512x512', 'casting_data']

In [4]:
os.listdir(os.path.join(path_to_dataset, "casting_data/casting_data"))

['train', 'test']

###Dividing into the Train and Test

The dataset was already dived into the Train and Test data in separate directories. Therefore, it is easy to use

In [5]:
BASE_DIR = os.path.join(path_to_dataset, "casting_data/casting_data")

TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

In [6]:
IMG_SIZE = (300, 300)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    labels = "inferred",
    label_mode = "binary",
    color_mode = "grayscale"
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    labels = "inferred",
    label_mode = "binary",
    color_mode = "grayscale"
    )

Found 6633 files belonging to 2 classes.
Found 715 files belonging to 2 classes.


In [7]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
    .cache()
    .shuffle(SHUFFLE_BUFFER_SIZE)
    .prefetch(PREFETCH_BUFFFER_SIZE)
)

test_dataset_final = (test_dataset
    .cache()
    .prefetch(PREFETCH_BUFFFER_SIZE)
)

###Model Implementation

In [8]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(300, 300, 1)),

    #Normalizing the pixels in the image
    tf.keras.layers.Rescaling(1./255),

    #Concolutional layers
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    #Flatterning Layer
    tf.keras.layers.GlobalAveragePooling2D(),

    #Dense Hiddent layer
    tf.keras.layers.Dense(64, activation="relu"),

    #output layer
    tf.keras.layers.Dense(1, activation="sigmoid")
])

cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 300, 300, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 298, 298, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,993 (394.50 KB)

 Trainable params: 100,993 (394.50 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
cnn_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Recall(name="recall")]
)

###Training the model

In [10]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "castinspect_best_model.keras",
    monitor="val_loss",
    save_best_only=True
)

In [11]:
history = cnn_model.fit(
    train_dataset_final,
    epochs=20,
    validation_data=test_dataset_final,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 792s 4s/step - accuracy: 0.5562 - loss: 0.6881 - recall: 0.0514 - val_accuracy: 0.6336 - val_loss: 0.6523 - val_recall: 0.0000e+00
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 757s 4s/step - accuracy: 0.6396 - loss: 0.6383 - recall: 0.3978 - val_accuracy: 0.8056 - val_loss: 0.4689 - val_recall: 0.6718
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 806s 4s/step - accuracy: 0.7691 - loss: 0.5165 - recall: 0.7178 - val_accuracy: 0.8182 - val_loss: 0.4235 - val_recall: 0.7977
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 771s 4s/step - accuracy: 0.8086 - loss: 0.4502 - recall: 0.7675 - val_accuracy: 0.7077 - val_loss: 0.5962 - val_recall: 0.9542
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 750s 4s/step - accuracy: 0.7960 - loss: 0.4617 - recall: 0.7442 - val_accuracy: 0.8434 - val_loss: 0.3808 - val_recall: 0.6908
Epoch 6/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 785s 4s/step - accuracy: 0.8249 - loss: 0.4232 - recall: 0.7736 - val_accuracy: 0.8238 - val_loss: 0.4294 - val_recall: 0

### Model Evaluation

In [14]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

loss, accuracy, recall = cnn_model.evaluate(test_dataset_final)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.8960 - loss: 0.2770 - recall: 0.8347
Loss: 0.2781234085559845
Accuracy: 0.8993006944656372
Recall: 0.8435114622116089


In [16]:
y_true = []
y_pred = []

for images, labels in test_dataset_final:
  preds = cnn_model.predict(images)
  y_true.extend(labels.numpy())
  y_pred.extend((preds > 0.5).astype(int))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 998ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 979ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 967ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step


In [17]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Confusion Matrix:
[[422  31]
 [ 41 221]]


In [18]:
print("\nDetailed Industry Report:")
print(classification_report(y_true, y_pred, target_names=['Defective', 'OK']))


Detailed Industry Report:
              precision    recall  f1-score   support

   Defective       0.91      0.93      0.92       453
          OK       0.88      0.84      0.86       262

    accuracy                           0.90       715
   macro avg       0.89      0.89      0.89       715
weighted avg       0.90      0.90      0.90       715

